# Arrow detection using tensorflow

## 1. Create folder hierarchy

In [20]:
%load_ext autoreload
%autoreload 2

import os
import pickle

files = pickle.load(open('files.pickle', 'rb'))
paths = pickle.load(open('paths.pickle', 'rb'))
paths

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{'WORKSPACE_PATH': 'Tensorflow/workspace',
 'SCRIPTS_PATH': 'Tensorflow/scripts',
 'APIMODEL_PATH': 'Tensorflow/models',
 'ANNOTATION_PATH': 'Tensorflow/workspace/annotations',
 'IMAGE_PATH': 'Tensorflow/workspace/images',
 'MODEL_PATH': 'Tensorflow/workspace/models',
 'PRETRAINED_MODEL_PATH': 'Tensorflow/workspace/pre-trained-models',
 'CHECKPOINT_PATH': 'Tensorflow/workspace/models/my_ssd_mobnet_640',
 'OUTPUT_PATH': 'Tensorflow/workspace/models/my_ssd_mobnet_640/export',
 'TFJS_PATH': 'Tensorflow/workspace/models/my_ssd_mobnet_640/tfjsexport',
 'TFLITE_PATH': 'Tensorflow/workspace/models/my_ssd_mobnet_640/tfliteexport',
 'PROTOC_PATH': 'Tensorflow/protoc'}

## 2. Create Label Map

In [3]:
labels = [{'name':'1', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## 3. Prepere test and train data

In [4]:
import glob

#DATA='/home/ox330/data/targets/all/'
#DATA='/home/ox330/data/targets/squares/'
#DATA='/home/ox330/data/targets/points/'
DATA='/home/vajicek/data/targets/points/'

annotation_jsons = glob.glob(DATA + '*.json')
test_size = int(len(annotation_jsons) / 3)
#test_size = int(len(annotation_jsons) - 10)
test = annotation_jsons[:test_size]
train = annotation_jsons[test_size:]

!mkdir -p {os.path.join(paths['IMAGE_PATH'], 'test')}
!mkdir -p {os.path.join(paths['IMAGE_PATH'], 'train')}

!rm {os.path.join(paths['IMAGE_PATH'], 'test')}/*
!rm {os.path.join(paths['IMAGE_PATH'], 'train')}/*

for test1 in test:
    !cp {test1} {os.path.join(paths['IMAGE_PATH'], 'test')}
    !cp {os.path.splitext(test1)[0]+'.jpg'} {os.path.join(paths['IMAGE_PATH'], 'test')}
for train1 in train:
    !cp {train1} {os.path.join(paths['IMAGE_PATH'], 'train')}
    !cp {os.path.splitext(train1)[0]+'.jpg'} {os.path.join(paths['IMAGE_PATH'], 'train')}

## 4. tfrecord

In [5]:
import generate_tfrecord as gen;
gen.json_to_tfrecord(os.path.join(paths['IMAGE_PATH'], 'train'), files['LABELMAP'],
    os.path.join(paths['IMAGE_PATH'], 'train'),
    os.path.join(paths['ANNOTATION_PATH'], 'train.record'), 40)
gen.json_to_tfrecord(os.path.join(paths['IMAGE_PATH'], 'test'), files['LABELMAP'],
    os.path.join(paths['IMAGE_PATH'], 'test'),
    os.path.join(paths['ANNOTATION_PATH'], 'test.record'), 40)

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


## 5. Copy Model Config to Training Folder

In [6]:
!cp {files['PRETRAINED_MODEL_CONFIG']} {paths['CHECKPOINT_PATH']}

## 6. Update Config For Transfer Learning

In [7]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [8]:
# config from pretrained model
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v1_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
  

In [9]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    text_format.Merge(f.read(), pipeline_config)

In [10]:
# update model training parameters
pipeline_config.model.ssd.num_classes = len(labels)
# pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.height = 512
# pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.width = 512

pipeline_config.train_config.batch_size = 6
pipeline_config.train_config.fine_tune_checkpoint = files['PRETRAINED_MODEL_CKP0']
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

pipeline_config.train_input_reader.label_map_path = files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'train.record')]

pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [
    os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [11]:
#store updated config
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

## 7. Train the Model

In [15]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [16]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [17]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(
    TRAINING_SCRIPT,
    paths['CHECKPOINT_PATH'],
    files['PIPELINE_CONFIG'])

In [18]:
!rm {paths['CHECKPOINT_PATH']}/ckpt-*
!rm {paths['CHECKPOINT_PATH']}/checkpoint

In [19]:
print(command)

python3 Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet_640 --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet_640/pipeline.config --num_train_steps=2000


In [ ]:
!{command}

## 8. Evaluate the Model

In [13]:
command = "python3 {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(
    TRAINING_SCRIPT,
    paths['CHECKPOINT_PATH'],
    files['PIPELINE_CONFIG'],
    paths['CHECKPOINT_PATH'])

In [14]:
print(command)

python3 Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_resnet50 --pipeline_config_path=Tensorflow/workspace/models/my_ssd_resnet50/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_ssd_resnet50


In [ ]:
!{command}

## 9. Convert to tf lite

In [11]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')
command = "python3 {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(
    FREEZE_SCRIPT,
    files['PIPELINE_CONFIG'],
    paths['CHECKPOINT_PATH'],
    paths['OUTPUT_PATH'])

print(FREEZE_SCRIPT)
print(files['PIPELINE_CONFIG'])
print(paths['CHECKPOINT_PATH'])
print(paths['OUTPUT_PATH'])
print(command)

Tensorflow/models/research/object_detection/exporter_main_v2.py 
Tensorflow/workspace/models/my_ssd_resnet50/pipeline.config
Tensorflow/workspace/models/my_ssd_resnet50
Tensorflow/workspace/models/my_ssd_resnet50/export
python3 Tensorflow/models/research/object_detection/exporter_main_v2.py  --input_type=image_tensor --pipeline_config_path=Tensorflow/workspace/models/my_ssd_resnet50/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/models/my_ssd_resnet50 --output_directory=Tensorflow/workspace/models/my_ssd_resnet50/export


In [8]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')
command = "python3 {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(
    TFLITE_SCRIPT,
    files['PIPELINE_CONFIG'],
    paths['CHECKPOINT_PATH'],
    paths['TFLITE_PATH'])
print(command)

python3 Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py  --pipeline_config_path=Tensorflow/workspace/models/my_ssd_resnet50/pipeline.config --trained_checkpoint_dir=Tensorflow/workspace/models/my_ssd_resnet50 --output_directory=Tensorflow/workspace/models/my_ssd_resnet50/tfliteexport


In [7]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')
command = "~/.local/bin/tflite_convert \
--saved_model_dir={} \
--output_file={}".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )

# --input_shapes=1,640,640,3 \
# --input_arrays=normalized_input_image_tensor \
# --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
# --inference_type=FLOAT \
# --allow_custom_ops
print(command)

~/.local/bin/tflite_convert --saved_model_dir=Tensorflow/workspace/models/my_ssd_resnet50/tfliteexport/saved_model --output_file=Tensorflow/workspace/models/my_ssd_resnet50/tfliteexport/saved_model/detect.tflite


In [9]:
!{command}

2022-01-01 14:04:10.374547: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-01 14:04:10.412689: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-01 14:04:10.412919: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-01 14:04:10.420084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-01 14:04:10.420301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Traceback (most recent call last):
  File "Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py", line 161, in <module>
    app.run(main)
  File "/home/vajicek/.local/lib/python3.8/site-packages/absl/app.py", line 303, in run
    _run_main(main, args)
  File "/home/vajicek/.local/lib/python3.8/site-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py", line 154, in main
    export_tflite_graph_lib_tf2.export_tflite_model(
  File "/home/vajicek/.local/lib/python3.8/site-packages/object_detection/export_tflite_graph_lib_tf2.py", line 366, in export_tflite_model
    concrete_function = detection_module.inference_fn.get_concrete_function(
  File "/home/vajicek/.local/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 1233, in get_concrete_function
    concrete = self._get_concrete_function_garbage_collected(*args, **kwargs)
  File "/home/vajicek/.local/